In [1]:
%matplotlib qt
import hyperspy.api as hs
import numpy
import matplotlib.pyplot as plt
import csv
plt.rcParams.update({'font.size': 20})
from hyperspy.misc.eels.tools import get_edges_near_energy

# Non-sparse Hyperspectrum

In [14]:
#file = numpy.loadtxt("1_0_1024_spimComplete", delimiter = ',', dtype=int)

ss = 32
how_many = 232
roll = 0

for slices in range(how_many):
    with open("results/"+str(how_many)+"_"+str(ss)+"_"+str(ss)+"_SpimComplete"+str(slices), "r") as f:
        reader = csv.reader(f, delimiter=',')
        for row in reader:
            full_array = numpy.asarray(row, dtype=float)
            full_array = numpy.reshape(full_array, (ss, ss, 1025))
            full_array = numpy.roll(full_array, roll, axis=0)
            s = hs.signals.Signal1D(full_array)
            s.save(f"results/slices/spim{slices}", overwrite=True)

In [15]:
s = hs.load("results/slices/*", stack=True)
s.set_signal_type("EELS")

[########################################] | 100% Completed |  0.1s


In [17]:
s.sum().plot()

# Sparse Hyperspectrum

In [5]:
#file = numpy.loadtxt("1_0_1024_spimComplete", delimiter = ',', dtype=int)

ss = 32
how_many = 232
roll = 0



for slices in range(how_many):
    f1 = open("results/232/"+str(how_many)+"_"+str(ss)+"_"+str(ss)+"_SparseSpimComplete"+str(slices), "r")
    r1 = csv.reader(f1, delimiter=',')
    
    f2 = open("results/232/"+str(how_many)+"_"+str(ss)+"_"+str(ss)+"_SparseSpimComplete_Hits"+str(slices), "r")
    r2 = csv.reader(f2, delimiter=',')
    
    for i in r1:
        for h in r2:
            spec = numpy.zeros(ss*ss*1025)
            i = numpy.asarray(i, dtype=int)
            h = numpy.asarray(h, dtype=int)
            spec[i] = h
            spec = numpy.reshape(spec, (ss, ss, 1025))
            s = hs.signals.Signal1D(spec)
            s.save(f"results/slices/spim{slices}", overwrite=True)

In [6]:
s = hs.load("results/slices/*", stack=True)
s.set_signal_type("EELS")

[########################################] | 100% Completed |  0.1s


In [7]:
s.plot()

# Sparse Hyperspectrum v2

In [137]:
file = numpy.fromfile("si_complete.txt", dtype='uint64')

ss = 512
channels = 1041
desired_frames = 50

file = numpy.mod(file, (ss*ss*channels*desired_frames))
unique, counts = numpy.unique(file, return_counts=True)
frames = int(numpy.max(unique / (ss * ss * channels))) + 1

print(f'Number of frames after binning is {frames}.')

Number of frames after binning is 22.


In [138]:
si = numpy.zeros(ss * ss * frames * channels, dtype='int8')
si[unique] += counts
si = numpy.reshape(si, (frames, ss, ss, channels))
#si = numpy.roll(si, 100, axis=1)

In [139]:
s = hs.signals.Signal1D(si)
s.sum(axis=2).plot()
s.inav[:, :, 0].plot()
s

<Signal1D, title: , dimensions: (512, 512, 22|1041)>